# make_dataset

sentencepieceでtokenizeしたID列をデータセットとしてpickleで保存する

文頭・文末を表す`bos`, `eos`トークンも追加する

|パラメータ名|内容|
|:--|:--|
| `THREAD_FILTER_TH_MIN_TOKEN` | トークン数がこのパラメータより小さいスレッドタイトルはデータセットに含めない |
| `THREAD_FILTER_TH_MAX_TOKEN` | トークン数がこのパラメータより大きいスレッドタイトルはデータセットに含めない |

## 入力ファイル

* `normalized_titles.txt`

## 出力ファイル

* `real_dataset.pickle`: 学習に利用するToken ID列

In [1]:
THREAD_FILTER_TH_MIN_TOKEN = 5
THREAD_FILTER_TH_MAX_TOKEN = 24

In [2]:
with open("normalized_titles.txt") as f:
    normalized_titles = [line.rstrip() for line in f]

In [3]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load('sentencepiece.model')

True

In [4]:
ids_list = []
for normalized in normalized_titles:
    ids = sp.encode_as_ids(normalized)
    ids_list.append([1] + ids + [2])  # <bos> <eos> トークンの付与

In [5]:
ids_tuple = [tuple(ids) for ids in ids_list]
deduped_ids_list = [list(ids) for ids in set(ids_tuple)]  # 重複排除（Tokenレベルでの重複排除）
deduped_ids_list = list(filter(lambda t: THREAD_FILTER_TH_MIN_TOKEN <= len(t) <= THREAD_FILTER_TH_MAX_TOKEN, deduped_ids_list))

In [6]:
deduped_ids_list[:3]

[[1, 81, 14, 9045, 554, 2199, 10331, 3, 2199, 2319, 89, 1248, 17, 2],
 [1,
  12477,
  6526,
  26,
  199,
  99,
  79,
  197,
  132,
  9860,
  4409,
  27,
  46,
  144,
  6614,
  2883,
  860,
  178,
  2],
 [1,
  135,
  9980,
  8137,
  21,
  8450,
  422,
  403,
  22,
  604,
  13048,
  118,
  207,
  2825,
  769,
  7737,
  737,
  2]]

In [ ]:
for ids in deduped_ids_list[:3]:
    print(sp.decode_ids(ids))

In [9]:
import pickle
with open("real_dataset.pickle", mode="wb") as f:
    pickle.dump(deduped_ids_list, f, protocol=pickle.HIGHEST_PROTOCOL)